In [46]:
import keras.src.callbacks as cb
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import tensorflow as tf
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Функция генерации датасета для логического выражения (a xor b) и (b xor c)
def generate_dataset(size: int=100) -> tuple[np.ndarray, np.ndarray]:
    """
    Generates a synthetic dataset for classification.

    Args:
        size (int): The number of samples in the dataset. Defaults to 100.

    Returns:
         tuple[np.ndarray, np.ndarray]: A tuple containing the feature matrix X and the target vector y.
            X is a 2D array where each row represents a sample and each column represents a feature.
            y is a 1D array where each element represents the class label of the corresponding sample.

    Examples:
        >>> X, y = generate_dataset(10)
        >>> print(X)
        [[0 0 0]
         [0 0 1]
         ...,
          [1 0 1],
          [0 1 0]]
        >>> print(y)
        [0 1 0 ..., 1 0 1]
    """
    X: np.ndarray = np.random.randint(0, 2, size=(size, 3))
    y: np.ndarray = np.logical_and(np.logical_not(X[:, 0] == X[:, 1]), np.logical_not(X[:, 1] == X[:, 2])).astype(int)
    return X, y

In [70]:
# Функция создания модели нейронной сети
def create_model() -> Sequential:
    """
    Creates a neural network model for binary classification.

    The model consists of 3 dense layers with ReLU activation function
    and sigmoid activation function in the output layer. It is compiled
    with binary cross-entropy loss, Adam optimizer and accuracy metric.

    Args:
        None

    Returns:
        Sequential: A compiled Keras neural network model for binary classification.

    Examples:
        >>> from tensorflow.keras.models import Sequential
        >>> baseline_model = create_model()
        >>> print(baseline_model.summary())
    """
    baseline_model: Sequential = Sequential()
    baseline_model.add(Input(shape=(3,), name='input'))
    baseline_model.add(Dense(1, activation='sigmoid'))
    
    baseline_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    baseline_model.summary()
    return baseline_model

In [75]:
def create_error_report(model_report: Sequential, val_split: float=0.2):
    data: pd.DataFrame = pd.read_csv('data/data_bin.csv')
    X = data.drop(columns=['target'])
    y = data['target']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    history: cb.History = model_report.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=val_split)
    history_df: pd.DataFrame = pd.DataFrame(history.history)

    fig, axes = plt.subplots(1, 2, figsize=(15,7))

    sns.lineplot(data=history_df['accuracy'], ax=axes[0])
    axes[0].set_title('val_ccuracy')

    sns.lineplot(data=history_df['loss'], ax=axes[1])
    axes[1].set_title('val_loss')

    model_report.evaluate(X_test, y_test)
    
    return model_report

In [66]:
def simulate_neural_layer(data_tensor, weight_tensor):
    """
    Simulates the behavior of a neural layer.
    
    Args:
        data_tensor: Input tensor with shape (5, 3)
        weight_tensor: Weight tensor
        
    Returns:
        Output tensor after applying the neural layer
    """

    # Тензор весов имеет размерность (3, n), где n - количество нейронов в слое
    num_neurons = weight_tensor.shape[-1]

    # Инициализируем output tensor с нулями по размеру 5 x n
    output_tensor = np.zeros((5, num_neurons))

    for i in range(5):
        for j in range(num_neurons):
            # Для каждого нейрона в слое вычисляем выход на основе входного тензора и веса
            output_tensor[i, j] += data_tensor[i, 0] * weight_tensor[0, j]
            output_tensor[i, j] += data_tensor[i, 1] * weight_tensor[1, j]
            output_tensor[i, j] += data_tensor[i, 2] * weight_tensor[2, j]

    return output_tensor

# Инициализация входных данных и весов
data_tensor = tf.random.uniform((5, 3))
weight_tensor = tf.random.uniform((3, 10))

# Симуляция поведения нейронного слоя
output_tensor = simulate_neural_layer(data_tensor, weight_tensor)

print("Входные данные:")
print(data_tensor)
print("\nТензор весов:")
print(weight_tensor)
print("\nВыход после симуляции:")
print(output_tensor)

Входные данные:
tf.Tensor(
[[0.44240892 0.8645688  0.39180982]
 [0.5129119  0.68290687 0.6594999 ]
 [0.04567504 0.28698313 0.6117815 ]
 [0.14929497 0.36755717 0.5202992 ]
 [0.47959077 0.8417324  0.27873743]], shape=(5, 3), dtype=float32)

Тензор весов:
tf.Tensor(
[[0.64196765 0.6231067  0.1494534  0.29552937 0.14258552 0.41521358
  0.75516474 0.12704074 0.6554433  0.8640796 ]
 [0.01536858 0.39732957 0.51612926 0.5122229  0.1909318  0.4733852
  0.32382703 0.7558397  0.87336016 0.756505  ]
 [0.8965746  0.6000954  0.6159599  0.8604013  0.33750594 0.54090583
  0.30035603 0.68966794 0.78704774 0.88780725]], shape=(3, 10), dtype=float32)

Выход после симуляции:
[[0.64858615 0.85431004 0.75368792 0.91071045 0.36039293 0.80490053
  0.73174483 0.9798981  1.35342693 1.38417876]
 [0.931059   0.98670077 0.83535016 1.06881559 0.42610759 0.8929733
  0.80656147 1.03616476 1.45166612 1.5453279 ]
 [0.58224016 0.50961453 0.53177953 0.68687522 0.26778668 0.48573464
  0.31117731 0.64464194 0.76207829 0.79

In [72]:
modelll = create_model()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_23 (Dense)                │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4 (16.00 B)

 Trainable params: 4 (16.00 B)

 Non-trainable params: 0 (0.00 B)

In [74]:
create_error_report(modelll)

FileNotFoundError: [Errno 2] No such file or directory: 'data/data_bin.csv'

In [69]:
modelll.get_layer('dense_22').get_weights()

[array([[ 0.8884676 ],
        [-0.27454597],
        [ 0.67404944]], dtype=float32),
 array([-0.09875816], dtype=float32)]

In [58]:
modelll.get_layer('dense_18').get_weights()

[array([[ 0.79077834, -0.42251903, -0.30026877, -0.14033717, -0.37014264],
        [-0.33898225, -0.13823019,  0.13702184, -0.1441328 ,  0.22989969],
        [ 0.8821523 ,  0.78911585,  0.472924  , -0.5663092 , -0.3922982 ]],
       dtype=float32),
 array([ 0.10029585,  0.11535221,  0.08029889,  0.        , -0.09838567],
       dtype=float32)]

In [32]:
X, y = generate_dataset(size=5)

In [33]:
X.shape

(5, 3)

In [34]:
X

array([[1, 0, 0],
       [1, 0, 1],
       [0, 0, 1],
       [0, 0, 0],
       [1, 0, 1]])